In [2]:
!pip install pytest-mock

In [3]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [4]:
%%writefile order_response_module.py
import asyncio

# Mock classes

class OrderResponseErrorCode:
    None_ = 0
    InvalidOrderStatus = 1
    InvalidNewOrderStatus = 2
    UnableToFindOrder = 3
    OrderQuantityZero = 4
    MissingSecurity = 5
    AlgorithmWarmingUp = 6

#Main OrderResponse Class
class OrderResponse:
    def __init__(self, order_id: int, error_code: int, error_message: str = None):
        self._order_id = order_id
        self._error_code = error_code
        self._error_message = error_message or "Default error message"

    @property
    def order_id(self):
        return self._order_id

    @property
    def error_message(self):
        return self._error_message if self.is_error else ""

    @property
    def error_code(self):
        return self._error_code

    @property
    def is_success(self):
        return self.is_processed and not self.is_error

    @property
    def is_error(self):
        return self.is_processed and self.error_code != OrderResponseErrorCode.None_

    @property
    def is_processed(self):
        return self != OrderResponse.unprocessed()

    @staticmethod
    def unprocessed():
        return OrderResponse(int(-1), OrderResponseErrorCode.None_, "Unprocessed order response")

    @staticmethod
    async def success(order_id):
        return  OrderResponse(order_id, OrderResponseErrorCode.None_)

    @staticmethod
    async def error(order_id, error_code, error_message):
        return await OrderResponse(order_id, error_code, error_message)

    @staticmethod
    async def invalid_status(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.InvalidOrderStatus, "Invalid order status")

    @staticmethod
    async def invalid_new_status(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.InvalidNewOrderStatus, "Invalid new order status")

    @staticmethod
    async def unable_to_find_order(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.UnableToFindOrder, "Unable to find order")

    @staticmethod
    async def zero_quantity(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.OrderQuantityZero, "Order quantity is zero")

    @staticmethod
    async def missing_security(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.MissingSecurity, "Missing security")

    @staticmethod
    async def warming_up(order_id):
        return await OrderResponse.error(order_id, OrderResponseErrorCode.AlgorithmWarmingUp, "Algorithm is warming up")



Overwriting order_response_module.py


In [8]:
%%writefile test_order_response_module.py
from unittest import mock
import pytest
from unittest.mock import AsyncMock
from order_response_module import OrderResponse, OrderResponseErrorCode 

@pytest.mark.asyncio
async def test_success_response():
    response = await OrderResponse.success(123)
    assert response.order_id == 123
    assert response.is_success is True
    assert response.is_error is False
    assert response.error_code == OrderResponseErrorCode.None_
    assert response.error_message == ""

@pytest.mark.asyncio
async def test_error_response(mocker):
    mock_response = OrderResponse(456, OrderResponseErrorCode.InvalidOrderStatus, "Error occurred")
    mocker.patch('order_response_module.OrderResponse.error', new_callable=AsyncMock, return_value=mock_response)

    response = await OrderResponse.error(456, OrderResponseErrorCode.InvalidOrderStatus, "Error occurred")
    assert response.order_id == 456
    assert response.is_success is False
    assert response.is_error is True
    assert response.error_code == OrderResponseErrorCode.InvalidOrderStatus
    assert response.error_message == "Error occurred"

@pytest.mark.asyncio
async def test_invalid_status_response(mocker):
    mock_response = OrderResponse(789, OrderResponseErrorCode.InvalidOrderStatus, "Invalid order status")
    mocker.patch('order_response_module.OrderResponse.invalid_status', new_callable=AsyncMock, return_value=mock_response)

    response = await OrderResponse.invalid_status(789)
    assert response.order_id == 789
    assert response.is_success is False
    assert response.is_error is True
    assert response.error_code == OrderResponseErrorCode.InvalidOrderStatus
    assert response.error_message == "Invalid order status"

@pytest.mark.asyncio
async def test_unable_to_find_order_response(mocker):
    mock_response = OrderResponse(101, OrderResponseErrorCode.UnableToFindOrder, "Unable to find order")
    mocker.patch('order_response_module.OrderResponse.unable_to_find_order', new_callable=AsyncMock, return_value=mock_response)

    response = await OrderResponse.unable_to_find_order(101)
    assert response.order_id == 101
    assert response.is_success is False
    assert response.is_error is True
    assert response.error_code == OrderResponseErrorCode.UnableToFindOrder
    assert response.error_message == "Unable to find order"

@pytest.mark.asyncio
async def test_zero_quantity_response(mocker):
    mock_response = OrderResponse(102, OrderResponseErrorCode.OrderQuantityZero, "Order quantity is zero")
    mocker.patch('order_response_module.OrderResponse.zero_quantity', new_callable=AsyncMock, return_value=mock_response)

    response = await OrderResponse.zero_quantity(102)
    assert response.order_id == 102
    assert response.is_success is False
    assert response.is_error is True
    assert response.error_code == OrderResponseErrorCode.OrderQuantityZero
    assert response.error_message == "Order quantity is zero"

@pytest.mark.asyncio
async def test_missing_security_response(mocker):
  mock_response = OrderResponse(103, OrderResponseErrorCode.MissingSecurity, "Missing security")
  mocker.patch('order_response_module.OrderResponse.missing_security', new_callable=AsyncMock, return_value=mock_response)
  response = await OrderResponse.missing_security(103)
  assert response.order_id == 103
  assert response.is_success is False
  assert response.is_error is True
  assert response.error_code == OrderResponseErrorCode.MissingSecurity
  assert response.error_message == "Missing security"

@pytest.mark.asyncio
async def test_warming_up_response(mocker):
    mock_response = OrderResponse(103, OrderResponseErrorCode.AlgorithmWarmingUp, "Algorithm is warming up")
    mocker.patch('order_response_module.OrderResponse.warming_up', new_callable=AsyncMock, return_value=mock_response)

    response = await OrderResponse.warming_up(103)
    assert response.order_id == 103
    assert response.is_success is False
    assert response.is_error is True
    assert response.error_code == OrderResponseErrorCode.AlgorithmWarmingUp
    assert response.error_message == "Algorithm is warming up"


Overwriting test_order_response_module.py


In [9]:
!pytest test_order_response_module.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 7 items                                                                   